In [2]:
import gym
import numpy as np

In [16]:
env = gym.make('CartPole-v0')

In [17]:
#parameters
obs_space = len(env.observation_space.high)
buckets = (20, 20, 20, 20)
LEARNING_RATE = 0.1
EPISODES = 99000
RENDER_RATE = 500
DISCOUNT = 0.95
MAX_FRAMES = 500
epsilon = 1
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES//2
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)
act_space = env.action_space.n
bucket_size = env.observation_space.high/buckets - env.observation_space.low/buckets
q_table = np.random.randint(low=0, high=10, size=(buckets + (act_space,)))

In [18]:
print(epsilon)

1


In [19]:
def bucket_no(state):
    bucket = state/bucket_size - env.observation_space.low/bucket_size
    return tuple(bucket.astype(np.int64))

In [20]:
def next_action(state):
    return np.argmax(q_table[bucket_no(state)])

In [21]:
def play():
    frame_min = 50
    epsilon = 1
    for episode in range(EPISODES):
        prev_state = env.reset()
        step = next_action(prev_state)
        curr_state, _, done, info = env.step(step)
        render = False
        if episode % RENDER_RATE == 0:
            render = True
        for frame in range(MAX_FRAMES):
            if render:
                env.render()
            qt_current = q_table[bucket_no(prev_state) + (step,)]
            max_qt_poss = np.max(q_table[bucket_no(curr_state)])
            new_q = (1 - LEARNING_RATE) * qt_current + LEARNING_RATE * (frame + DISCOUNT * max_qt_poss)
            q_table[bucket_no(prev_state) + (step,)] = new_q
            if np.random.random() > epsilon:
                step = next_action(curr_state)
            else:
                step = np.random.randint(0, env.action_space.n)
            prev_state = curr_state
            new_state, _, done, info = env.step(step)
            if done:
                if render:
                    print('episode = %d' % episode)
                    print(frame)
                break
        if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
            epsilon -= epsilon_decay_value
    env.close()

In [22]:
play()

episode = 0
33
episode = 500
26
episode = 1000
17
episode = 1500
15
episode = 2000
12
episode = 2500
25
episode = 3000
11
episode = 3500
13
episode = 4000
31
episode = 4500
15
episode = 5000
47
episode = 5500
39
episode = 6000
16
episode = 6500
8
episode = 7000
17
episode = 7500
14
episode = 8000
35
episode = 8500
51
episode = 9000
31
episode = 9500
20
episode = 10000
40
episode = 10500
25
episode = 11000
8
episode = 11500
13
episode = 12000
22
episode = 12500
20
episode = 13000
13
episode = 13500
29
episode = 14000
12
episode = 14500
25
episode = 15000
23
episode = 15500
10
episode = 16000
8
episode = 16500
23
episode = 17000
38
episode = 17500
35
episode = 18000
27
episode = 18500
25
episode = 19000
12
episode = 19500
9
episode = 20000
12
episode = 20500
7
episode = 21000
13
episode = 21500
14
episode = 22000
11
episode = 22500
15
episode = 23000
24
episode = 23500
35
episode = 24000
19
episode = 24500
28
episode = 25000
28
episode = 25500
12
episode = 26000
32
episode = 26500
22
epi

In [ ]:
import numpy as np

env = gym.make("CartPole-v0")

LEARNING_RATE = 0.2
frame_sum = 0

DISCOUNT = 0.95
EPISODES = 99000
SHOW_EVERY = 1000

DISCRETE_OS_SIZE = [10] * len(env.observation_space.high)
discrete_os_win_size = (env.observation_space.high/DISCRETE_OS_SIZE - env.observation_space.low/DISCRETE_OS_SIZE)

# Exploration settings
epsilon = 1  # not a constant, qoing to be decayed
START_EPSILON_DECAYING = 1
END_EPSILON_DECAYING = EPISODES*10
epsilon_decay_value = epsilon/(END_EPSILON_DECAYING - START_EPSILON_DECAYING)


q_table = np.random.uniform(low=0, high=10, size=(DISCRETE_OS_SIZE + [env.action_space.n]))


def get_discrete_state(state):
    discrete_state = (state - env.observation_space.low)/discrete_os_win_size
    return tuple(discrete_state.astype(np.int))  # we use this tuple to look up the 3 Q values for the available actions in the q-table


for episode in range(EPISODES):
    discrete_state = get_discrete_state(env.reset())
    done = False

    if episode % SHOW_EVERY == 0:
        render = True
        print(episode)
    else:
        render = False

    for frame in range(0,500):

        if np.random.random() > epsilon:
            # Get action from Q table
            action = np.argmax(q_table[discrete_state])
        else:
            # Get random action
            action = np.random.randint(0, env.action_space.n)


        new_state, __, done, _ = env.step(action)

        new_discrete_state = get_discrete_state(new_state)

        if episode % SHOW_EVERY == 0:
            env.render()
        #new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

        # If simulation did not end yet after last step - update Q table
        if not done:

            # Maximum possible Q value in next step (for new state)
            max_future_q = np.max(q_table[new_discrete_state])

            # Current Q value (for current state and performed action)
            current_q = q_table[discrete_state + (action,)]

            # And here's our equation for a new Q value for current state and action
            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * ( DISCOUNT * max_future_q)

            # Update Q table with new Q value
            q_table[discrete_state + (action,)] = new_q
        else:
            frame_sum += frame
            q_table[discrete_state + (action,)] = frame
            if render:
                print(f"frame={frame}")
                print(f"avg_frames={frame_sum/(episode+1)}")
            break

        discrete_state = new_discrete_state

    # Decaying is being done every episode if episode number is within decaying range
    if END_EPSILON_DECAYING >= episode >= START_EPSILON_DECAYING:
        epsilon -= epsilon_decay_value


env.close()

0
frame=23
avg_frames=23.0
1000
frame=13
avg_frames=21.865134865134866
2000
frame=27
avg_frames=21.55322338830585
3000
frame=24
avg_frames=21.60746417860713
4000
frame=17
avg_frames=21.640339915021244
5000
frame=20
avg_frames=21.66526694661068
6000
frame=10
avg_frames=21.60573237793701
7000
frame=22
avg_frames=21.48093129552921
8000
frame=9
avg_frames=21.41682289713786
9000
frame=20
avg_frames=21.397733585157205
10000
frame=17
avg_frames=21.364163583641634
11000
frame=10
avg_frames=21.264521407144805
12000
frame=20
avg_frames=21.209899175068745
13000
frame=19
avg_frames=21.24390431505269
14000
frame=36
avg_frames=21.210127848010856
15000
frame=17
avg_frames=21.18552096526898
16000
frame=12
avg_frames=21.200362477345166
17000
frame=18
avg_frames=21.21698723604494
18000
frame=16
avg_frames=21.23404255319149
19000
frame=38
avg_frames=21.235408662701964
20000
frame=12
avg_frames=21.22938853057347
21000
frame=13
avg_frames=21.223608399600018
22000
frame=19
avg_frames=21.24448888686878
23000

In [ ]:
if PLAY:
    import gym
    env = gym.make('CartPole-v1')
    score = input("Enter the score of the model you want to load ")
    model_type = input("Enter model type peak_model or avg_model ")
    trained_agent = load_model(f"{SAVE}/{model_type}-{TASK}-{score}.h5")
    state = env.reset()
    scores = []
    for _ in tqdm(range(PLAY_EPISODES)):
        env.reset()
        score_per_episode = 0
        while True:
            action = np.argmax(trained_agent.predict(np.array([state])))
            state, reward, done, info = env.step(action)
            score_per_episode += reward
            if SHOW:
                env.render()
            if done:
                scores.append(score_per_episode)
                break
        if PLOT:
            plot(scores)
    print(sum(scores)/PLAY_EPISODES)